In [1]:
# lr:0.001
# anneal_rate:0.9
# batch_size:32
# clip_norm:50
# num_epochs:5
# alpha:250
# beta:0
# max_beta:1
# step_beta:0.002
# anneal_iter:40000
# kl_anneal_iter:2000
# print_iter:100
# save_iter:5000
# num_workers:4

In [2]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [3]:
!pip install -q toolz
!pip install -q wandb

In [4]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

In [5]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [6]:
csv = pd.read_csv("ZINC_310k.csv")

In [7]:
smiles = csv['SMILES']

In [8]:
if 'runner.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner.xml', 'wb') as f:
        pickle.dump(runner, f)

In [9]:
with open('runner.xml', 'rb') as f:
    runner = pickle.load(f)

In [10]:
labels = torch.tensor(csv['LogP']).float()

In [11]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [12]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [13]:
N_TEST = 10000
VAL_FRAC = 0.05

In [14]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])


X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [15]:
train_loader = SemiMolTreeFolder(
    X_train,
    L_train,
    runner.vocab,
    conf["batch_size"],
    label_pct=0.5,
    num_workers=conf["num_workers"],
)

In [16]:
test_loader = SemiMolTreeFolderTest(
    X_test,
    L_test,
    runner.vocab,
    conf["batch_size"],
    num_workers=conf["num_workers"],
)

In [ ]:
print("Training model...")
runner.train_gen_pred(
    loader=train_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)

Training model...
Model #Params: 5207K
[100] Alpha: 250.000, Beta: 0.000, Loss: 286.09, KL: 69.47, MAE: 0.00712, Word Loss: 96.08, Topo Loss: 24.40, Assm Loss: 8.69, Pred Loss: 0.63, Word: 0.27, Topo: 0.80, Assm: 0.57, PNorm: 103.91, GNorm: 50.00
[Test] Alpha: 250.000, Beta: 0.000, Loss: 101.36, KL: 57.47, MAE: 0.00400, Word Loss: 36.76, Topo Loss: 7.62, Assm Loss: 4.76, Pred Loss: 0.21, Word: 0.41, Topo: 0.91, Assm: 0.51
[Test] Alpha: 250.000, Beta: 0.000, Loss: 116.44, KL: 57.59, MAE: 0.00551, Word Loss: 32.33, Topo Loss: 6.82, Assm Loss: 4.09, Pred Loss: 0.29, Word: 0.42, Topo: 0.90, Assm: 0.52
[Test] Alpha: 250.000, Beta: 0.000, Loss: 110.60, KL: 57.04, MAE: 0.00490, Word Loss: 32.87, Topo Loss: 8.03, Assm Loss: 4.30, Pred Loss: 0.26, Word: 0.40, Topo: 0.89, Assm: 0.57
[Test] Alpha: 250.000, Beta: 0.000, Loss: 92.10, KL: 58.80, MAE: 0.00415, Word Loss: 34.60, Topo Loss: 6.75, Assm Loss: 3.68, Pred Loss: 0.19, Word: 0.42, Topo: 0.91, Assm: 0.65
[Test] Alpha: 250.000, Beta: 0.000, Lo